In [123]:
import numpy as np

with open('kasyno10K.txt') as f:
    kasyno = np.array([int(x) for x in f.read()])

In [98]:
dice_types = {
    'normal': 0,
    'cheated': 1,
}
p_distribution = np.array(((np.ones(6) / 6) ,  (np.ones(6) / 10)))

p_distribution[1][-1] = 0.5

p_switch = np.array((0.04, 0.05))

def gen_data(size=10000):
    throws = []
    dices = []
    dice_type = dice_types['normal']
    for i in range(size):
        dices.append(dice_type)
        throws.append(
            np.random.choice(np.arange(1,7), p=p_distribution[dice_type])
        )
        
        if np.random.rand() <= p_switch[dice_type]:
                dice_type = 1 - dice_type
    return np.array(throws), np.array(dices)

def heuristic(data, interval, num_of_sixes):
    res = np.zeros(data.size, dtype=int)
    for i in range(0, len(data), interval):
        sample = data[i:i+interval]
        res[i:i+interval] = int((sample == 6).sum() >= num_of_sixes)
    return res

In [105]:
data, dices = gen_data(6000)
# data

In [106]:
heuristic_res = []
for interval in range(7, 30):
    j = interval // 2
    for num_of_sixes in range(j-4, j+5):
        val = (heuristic(data, interval, num_of_sixes) == dices).mean()
        heuristic_res.append([(interval, num_of_sixes), val])
x = np.array(heuristic_res)
ids = x[: ,1].argsort()
print('[(interval, num_of_sixes), result]')
x[ids[::-1]][:5]

[(interval, num_of_sixes), result]


array([[(21, 7), 0.8035],
       [(15, 5), 0.7973333333333333],
       [(21, 8), 0.7941666666666667],
       [(20, 7), 0.794],
       [(9, 3), 0.7935]], dtype=object)

In [107]:
def vitterbi(
    data,
    a=np.array([
        [1 - p_switch[0], p_switch[0]],
        [p_switch[1], 1 - p_switch[1]]
    ]),
    b=np.array([
        [p_distribution[0], p_distribution[0]],
        [p_distribution[1], p_distribution[1]]
    ]),
    pi=np.array([1,0]),
    ret_params=False
):
    alpha = np.zeros((2, data.size), dtype=np.float128)

    alpha[:, 0] = pi
    for t in range(1, data.size):
        for j in range(2):
            for i in range(2):
                alpha[j, t] += alpha[i, t-1] * a[i, j] * b[i, j, data[t-1] - 1]
            alpha[j, t] *= 1
    
    beta = np.zeros((2, data.size), dtype=np.float128)
    beta[:, data.size - 1] = 1
    for t in range(data.size - 2, -1, -1):
        for j in range(2):
            for i in range(2):
                beta[j, t] += beta[i, t+1] * a[i, j] * b[i, j, data[t] - 1]
            beta[j, t] *= 1
    
    gamma = alpha * beta / (alpha * beta).sum() 
    
    if ret_params:
        return (alpha, beta, gamma)
    return alpha.argmax(0), beta.argmax(0), gamma.argmax(0)
        
x = vitterbi(data)
for x, name in zip(x, ['alpha', 'beta', 'gamma']):
    print('model:', name)
    print('sum:', x.sum())
    print('mean:', (x == dices).mean())
    print('-----------------')

model: alpha
sum: 2536
mean: 0.7526666666666667
-----------------
model: beta
sum: 2587
mean: 0.7685
-----------------
model: gamma
sum: 2435
mean: 0.8338333333333333
-----------------


# Excersize 4
#### We don't know model parameters

In [171]:
def count_ksi(data, alpha, a, b, betha):
    ksi = np.zeros((data.size - 1, 2, 2 ), dtype=np.float128)
    for t in range(data.size - 1):
        for i in range(2):
            for j in range(2):
                ksi[t, i, j] = (
                    alpha[i, t] *
                    a[i, j] *
                    b[i, j, (data[t+1] - 1)] *
                    betha[j, t+1]
                )
    return ksi

def EM(data, a, b, pi, iter_num=5):
    a = a.copy()
    b = b.copy()
    for SIZE in range(iter_num):
        alpha, betha, gamma = vitterbi(data, a, b, pi, True)
        ksi = count_ksi(data, alpha, a, b, betha)
        a_roof = np.zeros(a.shape, dtype=np.float128)
        b_roof = np.zeros(b.shape, dtype=np.float128)
        for i in range(2):
            for j in range(2):
                a[i, j] = ksi[:, i, j].sum() / ksi[:, i, :].sum()
        for i in range(2):
            for k in range(6):
                b[i, :, k] = gamma[i, (data - 1) == k].sum() / gamma[i, :].sum() 
    
    return a, b[:, 0]

p_switch = np.random.rand(2)

for i in range(2):
    p_distribution[i] = np.random.rand(6)
    p_distribution[i] /= p_distribution[i].sum()
print(p_switch)
print(p_distribution)
a=np.array([
    [1 - p_switch[0], p_switch[0]],
    [p_switch[1], 1 - p_switch[1]]
])
b=np.array([
    [p_distribution[0], p_distribution[0]],
    [p_distribution[1], p_distribution[1]]
])
pi=np.array([1,0])



t = EM(data[:6000], a, b, pi, 20)
print(
    t[0].sum(1),
    t[1].sum(1)
)

print()
t

[0.15284744 0.19141021]
[[0.01821802 0.10076424 0.14973304 0.08835911 0.33814757 0.30477802]
 [0.14043395 0.06796315 0.21368773 0.06804363 0.25836263 0.25150891]]
[1. 1.] [1. 1.]



(array([[1.00000000e+00, 3.56667195e-73],
        [1.00000000e+00, 2.21721065e-67]]),
 array([[1.44518295e-01, 1.29183020e-01, 1.36517282e-01, 1.40184413e-01,
         1.33016839e-01, 3.16580152e-01],
        [3.67106012e-65, 3.28151279e-65, 3.46781803e-65, 3.55673645e-65,
         3.37889962e-65, 1.00000000e+00]]))

In [151]:
for i in range(2):
    p_distribution[i] = np.random.rand(6)
    p_distribution[i] /= p_distribution[i].sum()
a=np.array([
    [1 - p_switch[0], p_switch[0]],
    [p_switch[1], 1 - p_switch[1]]
])
b=np.array([
    [p_distribution[0], p_distribution[0]],
    [p_distribution[1], p_distribution[1]]
])
pi=np.array([1,0])

In [162]:
print(p_switch)
print(p_distribution)
t = EM(kasyno[:6000], a, b, pi, iter_num=2)
print(
    t[0].sum(1),
    t[1].sum(1)
)

print()
t

[0.64025303 0.12562101]
[[0.07214421 0.20219166 0.27743459 0.22564479 0.07397163 0.14861312]
 [0.28624503 0.04311781 0.299834   0.10703876 0.10863082 0.15513359]]
[1. 1.] [1. 1.]



(array([[3.65235443e-03, 9.96347646e-01],
        [8.28972536e-04, 9.99171027e-01]]),
 array([[0.13349818, 0.13760741, 0.18185745, 0.09328206, 0.17186961,
         0.28188529],
        [0.13642194, 0.13336988, 0.17823306, 0.09298958, 0.17252174,
         0.28646379]]))